In [ ]:
#피어슨 상관분석
import pandas as pd

intent_df = pd.read_csv("지능형 기기 이용 의도_VRAR기기.csv", encoding="cp949")
frequency_df = pd.read_csv("지능형 기기 이용빈도_VRAR기기.csv", encoding="cp949")

# 첫 번째 행을 컬럼명으로 지정
intent_df.columns = intent_df.iloc[0]
intent_df = intent_df[1:].reset_index(drop=True)

frequency_df.columns = frequency_df.iloc[0]
frequency_df = frequency_df[1:].reset_index(drop=True)

# 이용 의도 점수 변환 
intent_weights = [1, 2, 3, 6, 0]  # "전혀 없다" ~ "매우 있다", "잘 모르겠다"(0점 처리)
intent_columns = ['전혀 없다 (%)', '별로 없다 (%)', '약간 있다 (%)', '매우 있다 (%)', '잘 모르겠다 (%)']
intent_df['이용 의도 점수'] = (intent_df[intent_columns].astype(float) * intent_weights).sum(axis=1) / 100

# 이용 빈도 점수 변환
frequency_weights = [30, 10, 5, 2, -5, -10]  
frequency_columns = ['거의 매일 (%)', '일주일에 한두 번 (%)', '1개월에 한두 번 (%)', '6개월에 한두 번 (%)', '최근 1년간 이용 한적 없음 (%)', '전혀 이용한 적 없음 (%)']
frequency_df["이용 빈도 점수"] = (frequency_df[frequency_columns].astype(float) * frequency_weights).sum(axis=1) / sum(frequency_weights)

# 필요한 컬럼 선택
intent_df = intent_df[['구분별(1)', '구분별(2)', '이용 의도 점수']]
frequency_df = frequency_df[['구분별(1)', '구분별(2)', '이용 빈도 점수']]

# 병합
merged_df = pd.merge(intent_df, frequency_df, on=['구분별(1)', '구분별(2)'])

# 데이터 타입 변환
merged_df['이용 의도 점수'] = merged_df['이용 의도 점수'].astype(float)
merged_df['이용 빈도 점수'] = merged_df['이용 빈도 점수'].astype(float)

# 그룹별 피어슨 상관계수 계산
group_correlations = {}

unique_groups = merged_df["구분별(1)"].unique()

for group in unique_groups:
    subset = merged_df[merged_df["구분별(1)"] == group]
    if len(subset) > 1:  # 최소한 두 개 이상의 데이터 필요
        correlation = subset[['이용 의도 점수', '이용 빈도 점수']].corr(method='pearson').iloc[0, 1]
        group_correlations[group] = correlation

# 결과 출력
print(group_correlations)